In [8]:
# Imorting libraries 
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import chart_studio.plotly as py
import seaborn as sns
import plotly.express as px
import math
import cufflinks as cf
import plotly.figure_factory as ff
import scipy
import plotly.graph_objects as go
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
from datetime import timedelta
from subprocess import check_output
import matplotlib as plt
from pathlib import Path
from plotly.subplots import make_subplots

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

cf.go_offline()


In [9]:
city_dict = pd.read_csv("/Users/akimovh/Library/CloudStorage/GoogleDrive-akimovhresearch@gmail.com/My Drive/predatory pricing/data/dict/L_CITY_MARKET_ID.csv")

# HI market

In [ ]:
# Convert base path to Path object for better path handling
base_path = Path("/Users/akimovh/Library/CloudStorage/GoogleDrive-akimovhresearch@gmail.com/My Drive/predatory pricing/data/DB1B")
    
# Pre-allocate list for DataFrames
all_tickets_HI = []
all_tickets_HA = []

for year in range(2014, 2025):
      for q in range(1, 5):
            # Skip 2024 Q4
            if year == 2024 and q == 4:
                 continue
            # Define file paths
            ticket_file = base_path / f"Ticket/{year}_{q}.csv"
            market_file = base_path / f"Market/Origin_and_Destination_Survey_DB1BMarket_{year}_{q}.csv"

            market_cols = ['ItinID', 'TkCarrier', 'OpCarrier', 'MktCoupons', 
                         'OriginCityMarketID', 'DestCityMarketID', 'OriginCountry',
                        'OriginState', 'DestCountry', 'DestState']
            
            market = pd.read_csv(market_file, usecols=market_cols)
            mask_HI = (
                        (market['MktCoupons'] == 1) &
                        ((market['OriginState'] == 'HI') or (market['DestState'] == 'HI')) 
                      )
            
            mask_HA = (
                        (market['MktCoupons'] == 1) &
                        (market['TkCarrier'] == 'HA') 
                      )
            market_HI = market[mask_HI]
            market_HA = market[mask_HA]


            Itin_ID_HI = market_HI.ItinID.unique()
            Itin_ID_HI = market_HA.ItinID.unique()
            

            
            # Read ticket data with only necessary columns
            ticket_cols = ['ItinID', 'Coupons', 'RoundTrip', 'DollarCred', 'Year',
                           'Quarter', 'ItinFare', 'BulkFare', 'Distance', 'DistanceGroup',
                           'Passengers', 'OnLine']
            

            ticket = pd.read_csv(ticket_file, usecols=ticket_cols)
            mask_HI = (
                        (ticket['ItinID'].isin(Itin_ID_HI)) &
                        (ticket['OnLine'] == 1)
                      )
            
            mask_HA = (
                        (ticket['ItinID'].isin(Itin_ID_HA)) &
                        (ticket['OnLine'] == 1)
                      )
            ticket_HI = ticket[mask_HI]
            ticket_HA = ticket[mask_HA]

            # Add city names

            market_HI = pd.merge(market_HI, city_dict,  left_on = 'OriginCityMarketID', right_on = 'Code', how = 'left')
            ticket_HI = pd.merge(market_HI, city_dict,  left_on = 'DestCityMarketID', right_on = 'Code', how = 'left')
            ticket_HA = pd.merge(ticket_HA, city_dict,  left_on = 'OriginCityMarketID', right_on = 'Code', how = 'left')
            ticket_HA = pd.merge(ticket_HA, city_dict,  left_on = 'DestCityMarketID', right_on = 'Code', how = 'left')

            
          # Create msa_pair using vectorized operations

            market_HI['city_pair_non_direct'] = market_HI.apply(
                lambda row: str(tuple(sorted([row['Description_x'], 
                                           row['Description_y']]))), 
                axis=1
            )
            market_HI['state_pair_non_direct'] = market_HI.apply(
                lambda row: str(tuple(sorted([row['OriginState'], 
                                           row['DestState']]))), 
                axis=1
            )

            market_HI['city_pair_direct'] = market_HI.apply(
                lambda row: str(tuple(([row['Description_x'], 
                                           row['Description_y']]))), 
                axis=1
            )
            market_HI['state_pair_direct'] = market_HI.apply(
                lambda row: str(tuple(([row['OriginState'], 
                                           row['DestState']]))), 
                axis=1
            )

            market_HA['city_pair_non_direct'] = market_HA.apply(
                lambda row: str(tuple(sorted([row['Description_x'], 
                                           row['Description_y']]))), 
                axis=1
            )
            market_HA['state_pair_non_direct'] = market_HA.apply(
                lambda row: str(tuple(sorted([row['OriginState'], 
                                           row['DestState']]))), 
                axis=1
            )

            market_HA['city_pair_direct'] = market_HA.apply(
                lambda row: str(tuple(([row['Description_x'], 
                                           row['Description_y']]))), 
                axis=1
            )
            market_HA['state_pair_direct'] = market_HA.apply(
                lambda row: str(tuple(([row['OriginState'], 
                                           row['DestState']]))), 
                axis=1
            )
            
            # Select final columns and drop duplicates
            market_HI = market_HI[['ItinID', 'TkCarrier', 'city_pair', 'OpCarrier', 'state_pair']].drop_duplicates()
            market_HA = market_HA[['ItinID', 'TkCarrier', 'city_pair', 'OpCarrier', 'state_pair']].drop_duplicates()

            
            # # Merge and append to results
            merged_HI = pd.merge(market_HI, ticket_HI, on='ItinID')
            merged_HA = pd.merge(merged_HA, ticket_HA, on='ItinID')

            all_tickets_HI.append(merged_HI)
            all_tickets_HA.append(merged_HA)
            
df_HI = pd.concat(all_tickets_HI)
df_HA = pd.concat(all_tickets_HA)
df_HI['date'] = pd.to_datetime(df_HI['Year'].astype(str) + '-' + ((df_HI['Quarter'] - 1) * 3 + 1).astype(str) + '-01')
df_HA['date'] = pd.to_datetime(df_HA['Year'].astype(str) + '-' + ((df_HA['Quarter'] - 1) * 3 + 1).astype(str) + '-01')


In [44]:
conditions = [
    tickets['state_pair'] == "('HI', 'HI')",
    tickets['country_pair'] == "('US', 'US')",
]
choices = ['Interstate', 'Other domestic']

tickets['flight_type'] = np.select(conditions, choices, default='Bad data')

In [ ]:
plot_data = tickets.groupby(by = ['date', 'flight_type'], as_index = False).Passengers.sum()
plot_data['Passengers'] = plot_data['Passengers']*10
fig = px.line(plot_data, x="date", y="Passengers", color='flight_type')
fig.show()

In [ ]:
plot_data = tickets.groupby(by = ['date', 'flight_type'], as_index = False).Passengers.sum()
plot_data['total_Passengers'] = plot_data.groupby(by = ['date'], as_index = False).Passengers.transform('sum')
plot_data['Share'] = plot_data['Passengers']/plot_data['total_Passengers']*100
fig = px.line(plot_data, x="date", y="Share", color='flight_type')
fig.show()

## Interisland flights

In [73]:
inner_ha = tickets[tickets['state_pair'] == "('HI', 'HI')"].query("TkCarrier in ['HA', 'WN', 'WP']")

In [ ]:
plot_data = inner_ha.groupby(by = ['date', 'TkCarrier'], as_index = False).Passengers.sum()
plot_data['Passengers'] = plot_data['Passengers']*10
fig = px.line(plot_data, x="date", y="Passengers", color='TkCarrier')
fig.show()

In [ ]:
plot_data = inner_ha.groupby(by = ['date', 'TkCarrier'], as_index = False).Passengers.sum()
plot_data['Passengers'] = plot_data['Passengers']*10
plot_data['total_Passengers'] = plot_data.groupby(by = ['date'], as_index = False).Passengers.transform('sum')
plot_data['Share'] = plot_data['Passengers']/plot_data['total_Passengers']*100
fig = px.line(plot_data, x="date", y="Share", color='TkCarrier')
fig.show()

In [113]:
inner_ha_agg = inner_ha.groupby(by = ['TkCarrier', 'date', 'city_pair'], as_index = False).Passengers.sum()

In [114]:
inner_ha_fares = inner_ha.query('ItinFare>20')
inner_ha_fares['passengers_per_quarter'] = inner_ha_fares.groupby(by = ['TkCarrier', 'city_pair', 'date']).Passengers.transform('sum')


In [115]:
inner_ha_fares = inner_ha_fares.sort_values(by = 'ItinFare')
inner_ha_fares['passengers_per_quarter_cum'] = inner_ha_fares.groupby(by = ['TkCarrier', 'city_pair', 'date']).Passengers.transform('cumsum')
mask = (
        (inner_ha_fares['passengers_per_quarter_cum']/inner_ha_fares['passengers_per_quarter']>0.01) & 
        (inner_ha_fares['passengers_per_quarter_cum']/inner_ha_fares['passengers_per_quarter']<0.99) 
                   )
inner_ha_fares = inner_ha_fares[mask]

In [116]:
inner_ha_fares['Revenue'] = inner_ha_fares['Passengers']*inner_ha_fares['ItinFare']
inner_ha_fares = inner_ha_fares.groupby(by = ['TkCarrier', 'date', 'city_pair'], as_index = False).agg(Passengers = ('Passengers', 'sum'),
                                                                                                       Revenue = ('Revenue', 'sum'))
inner_ha_fares_agg = inner_ha_fares.groupby(by = ['TkCarrier', 'date'], as_index = False).agg(Passengers = ('Passengers', 'sum'),
                                                                                                       Revenue = ('Revenue', 'sum'))

In [118]:
inner_ha_fares['Price'] = inner_ha_fares.Revenue/ inner_ha_fares.Passengers
inner_ha_fares_agg['Price'] = inner_ha_fares_agg.Revenue/ inner_ha_fares_agg.Passengers

In [ ]:
plot_data = inner_ha_fares_agg.groupby(by = ['date', 'TkCarrier'], as_index = False).Price.sum()
fig = px.line(plot_data, x="date", y="Price", color='TkCarrier')
fig.show()

In [21]:
conditions = [
    tickets['state_pair'] == "('HI', 'HI')",
    tickets['country_pair'] == "('US', 'US')",
]
choices = ['Interstate', 'Other domestic']

tickets['flight_type'] = np.select(conditions, choices, default='Bad data')

In [23]:
tickets.to_csv("/Users/akimovh/My documents/projects/predatory pricing/data/temporal/HI_flights.csv")

# Hawaiian airlines

In [121]:
# Convert base path to Path object for better path handling
base_path = Path("/Users/akimovh/Library/CloudStorage/GoogleDrive-akimovhresearch@gmail.com/My Drive/predatory pricing/data/DB1B") 
   
# Pre-allocate list for DataFrames
all_tickets = []

for year in range(2017, 2025):
      for q in range(1, 5):
            # Skip 2024 Q4
            if year == 2024 and q == 4:
                 continue
            # Define file paths
            ticket_file = base_path / f"Ticket/{year}_{q}.csv"
            market_file = base_path / f"Market/Origin_and_Destination_Survey_DB1BMarket_{year}_{q}.csv"

            market_cols = ['ItinID', 'TkCarrier', 'OpCarrier', 'MktCoupons', 
                         'OriginCityMarketID', 'DestCityMarketID', 'OriginCountry',
                        'OriginState', 'DestCountry', 'DestState']
            
            tmp_1 = pd.read_csv(market_file, usecols=market_cols)
            tmp_1 = tmp_1.query("MktCoupons == 1 and TkCarrier == 'HA'")
            Itin_ID = tmp_1.ItinID.unique()
            

            
            # Read ticket data with only necessary columns
            ticket_cols = ['ItinID', 'Coupons', 'RoundTrip', 'DollarCred', 'Year',
                           'Quarter', 'ItinFare', 'BulkFare', 'Distance', 'DistanceGroup',
                           'Passengers', 'OnLine']
            
            tmp = pd.read_csv(ticket_file, usecols=ticket_cols)
            
            # Apply filters using vectorized operations
            mask = (tmp['ItinID'].isin(Itin_ID)) & (tmp['DollarCred'] == 1) & (tmp['BulkFare'] == 0) & (tmp['OnLine'] == 1)
            tmp = tmp[mask]

            # Add city names

            tmp_1 = pd.merge(tmp_1, city_dict,  left_on = 'OriginCityMarketID', right_on = 'Code', how = 'left')
            tmp_1 = pd.merge(tmp_1, city_dict,  left_on = 'DestCityMarketID', right_on = 'Code', how = 'left')

            
          # Create msa_pair using vectorized operations

            tmp_1['city_pair'] = tmp_1.apply(
                lambda row: str(tuple(sorted([row['Description_x'], 
                                           row['Description_y']]))), 
                axis=1
            )
            tmp_1['state_pair'] = tmp_1.apply(
                lambda row: str(tuple(sorted([row['OriginState'], 
                                           row['DestState']]))), 
                axis=1
            )
            tmp_1['country_pair'] = tmp_1.apply(
                lambda row: str(tuple(sorted([row['OriginCountry'], 
                                           row['DestCountry']]))), 
                axis=1
            )
            
            # Select final columns and drop duplicates
            tmp_1 = tmp_1[['ItinID', 'TkCarrier', 'city_pair', 'OpCarrier', 'state_pair', 'country_pair']].drop_duplicates()

            
            # # Merge and append to results
            merged = pd.merge(tmp_1, tmp, on='ItinID')
            all_tickets.append(merged)
            
tickets = pd.concat(all_tickets)

In [125]:
conditions = [
    tickets['state_pair'] == "('HI', 'HI')",
    tickets['state_pair'].str.contains("'HI'"),
]
choices = ['Interstate HI', 'Hi-Mainland']

tickets['flight_type'] = np.select(conditions, choices, default='Other domestic')

In [126]:
tickets['date'] = pd.to_datetime(tickets['Year'].astype(str) + '-' + ((tickets['Quarter'] - 1) * 3 + 1).astype(str) + '-01')

In [ ]:
plot_data = tickets.query("flight_type in ['Interstate HI', 'Hi-Mainland']").groupby(by = ['date', 'flight_type'], as_index = False).Passengers.sum()
plot_data['Passengers'] = plot_data['Passengers']*10
fig = px.line(plot_data, x="date", y="Passengers", color='flight_type')
fig.show()

In [ ]:
plot_data = tickets.query("flight_type in ['Interstate HI', 'Hi-Mainland']").groupby(by = ['date', 'flight_type'], as_index = False).Passengers.sum()
plot_data['Passengers'] = plot_data['Passengers']*10
plot_data['total_Passengers'] = plot_data.groupby(by = ['date'], as_index = False).Passengers.transform('sum')
plot_data['Share'] = plot_data['Passengers']/plot_data['total_Passengers']*100
fig = px.line(plot_data, x="date", y="Share", color =  'flight_type' )
fig.show()

In [144]:
costs = pd.DataFrame()
for year in range(2017,2025):
    costs = pd.concat([costs, pd.read_csv(f"/Users/akimovh/My documents/projects/predatory pricing/data/cost_data/{year}.csv")])
costs = costs.query("UNIQUE_CARRIER == 'HA'")
costs = costs.query("REGION == 'D'")
costs['date'] = pd.to_datetime(costs['YEAR'].astype(str) + '-' + ((costs['QUARTER'] - 1) * 3 + 1).astype(str) + '-01')


In [153]:
costs = costs.sort_values(by = 'date')


In [ ]:
plot_data = costs[['date', "INCOME_PRE_TAX"]]
plot_data["INCOME_PRE_TAX"] = plot_data["INCOME_PRE_TAX"]*1000
fig = px.line(costs, x="date", y="INCOME_PRE_TAX")
fig.show()